In [ ]:
!pip install statsmodels

In [ ]:
#import needed packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from google.colab import drive
#mounting my google driving to access data
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#converting the csv synthetic_data to a pandas data frame
df = pd.read_csv('/content/drive/MyDrive/Second Year Project /2021-2228_anon_cleaned.csv')

#converting the xlsx data to a pandas data frame
df2 = pd.read_excel("/content/drive/MyDrive/Second Year Project /Master LA _ Mentor Database.xlsx")

<ipython-input-3-ee2e44619d2d>:2: DtypeWarning: Columns (7,8,9,10,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Second Year Project /2021-2228_anon_cleaned.csv')


In [ ]:

# Add a new column 'LA' with zeros to df
df['LA'] = 0

#Change the name of the term column in the LA data set to upper case to match df.
df2 = df2.rename(columns={'term': 'Term'})
#Merge the dfs on the specified columns
merged_df = pd.merge(df, df2, on=['subject', 'catalog_nbr', 'class_section', 'Term'], how='inner', suffixes=('', '_df2'))
# Update the 'LA' column based on the condition
df.loc[merged_df.index, 'LA'] = 1  # Set the 'LA' column to 1 where the rows match
# keeping the 'LA' column as integers
df['LA'] = df['LA'].astype(int)



In [ ]:
# Create a new column 'gender dummy' with 1 for male or 0 for else
df['gender dummy'] = df['gender'].apply(lambda x: 1 if x in ['F', 'U'] else 0)


# Create a new column 'First Gen dummy' with 1 for First Generation students or 0 for else
df['First Gen dummy'] = df['First Generation'].apply(lambda x: 1 if x == 'First Gen' else 0)
# Fill NaN values with 0
df['First Generation'] = df['First Generation'].fillna(0)

# Create a new column 'AALANA dummy' with 1 for AALANA or 0 for else
df['AALANA dummy'] = df['AALANA'].apply(lambda x: 1 if x == 'AALANA' else 0)

# Create a new column 'DFW dummy' with 1 for 'D' or 'F' or 'W' grade or lower or 0 for else
df['DFW dummy'] = df['Grade'].apply(lambda x: 1 if x in ['D', 'F', 'W'] else 0)
# If you want to add 'NP' for Not Passed, you can modify the condition as follows:
# df['DFW dummy'] = df['Grade'].apply(lambda x: 1 if x in ['D', 'F', 'W', 'NP'] else 0)




In [ ]:
# Rename the 'gender' column to 'Gender'
df = df.rename(columns={'gender': 'Gender'})
# Map values in the 'Gender' column
gender_mapping = {'M': 'Male', 'F': 'Female', 'U': 'Unknown'}
df['Gender'] = df['Gender'].map(gender_mapping)

df = df.rename(columns={'AALANA': 'Race'})
# Map values in the 'Race' column
df['Race'] = df['Race'].apply(lambda x: 'AALANA' if x == 'AALANA' else 'Non AALANA')

# Rename the 'First Generation' column to 'First Gen status'
df = df.rename(columns={'First Generation': 'First Gen status'})
# Map values in the 'First Gen status' column
df['First Gen status'] = df['First Gen status'].apply(lambda x: 'FG' if x == 'First Gen' else 'CG')


# Rename the 'LA' column to 'Instructional type'
df = df.rename(columns={'LA': 'Instruction type'})
# Map values in the 'Instructional type' column
df['Instruction type'] = df['Instruction type'].apply(lambda x: 'LA' if x == 1 else 'Traditional')

In [ ]:

# Create a cross-tabulation table
table = pd.crosstab(index=[df['Gender'], df['Race'], df['First Gen status'], df['Instruction type']],
                    columns='N',
                    margins=True, margins_name='Total', rownames=['Gender', 'Race', 'First Gen status', 'Instruction type'],
                    colnames=['N'])

# Convert the table to LaTeX format
latex_table = table.to_latex()

# Print the LaTeX table
print(latex_table)

#print(table)



\begin{tabular}{llllrr}
\toprule
      &            &    & N &        N &    Total \\
Gender & Race & First Gen status & Instruction type &          &          \\
\midrule
Female & AALANA & CG & LA &     1450 &     1450 \\
      &            &    & Traditional &   126628 &   126628 \\
      &            & FG & LA &        7 &        7 \\
      &            &    & Traditional &    18878 &    18878 \\
      & Non AALANA & CG & LA &    12288 &    12288 \\
      &            &    & Traditional &   982648 &   982648 \\
      &            & FG & LA &       15 &       15 \\
      &            &    & Traditional &    63904 &    63904 \\
Male & AALANA & CG & LA &     2199 &     2199 \\
      &            &    & Traditional &   199454 &   199454 \\
      &            & FG & Traditional &    29854 &    29854 \\
      & Non AALANA & CG & LA &    24213 &    24213 \\
      &            &    & Traditional &  1959195 &  1959195 \\
      &            & FG & LA &        4 &        4 \\
      &          

<ipython-input-7-863e67c76f3a>:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = table.to_latex()


In [ ]:
# Print counts of different elements in the 'First Gen status' column
counts = df2['Course Title'].value_counts()
print(counts)

FYW: Writing Seminar              26072
Introduction to Psychology        16173
College Physics I                 15421
Principles of Microeconomics      14322
Introduction to Statistics I      12867
                                  ...  
Furniture Des Graduate Studio         1
User Experience Design                1
IntrotoHealthInformatics              1
FoundationsOfWebTechnologies I        1
Propaganda & Photography              1
Name: Course Title, Length: 6904, dtype: int64


In [ ]:
# Filtering based on conditions
filtered_df = df[(df['Gender'] == 'Female') & (df['Race'] == 'AALANA') & (df['First Gen status'] == 'CG') & (df['Instruction type'] == 'LA')]

# Counting the number of elements in the 'courses' column
num_elements = filtered_df['Course Title'].nunique()

num_elements

5

In [ ]:
unique_elements = df['catalog_nbr'].value_counts()


print("Unique elements in 'catalog_nbr' column:")
print(unique_elements)

Unique elements in 'catalog_nbr' column:
101    85099
210    53428
211    50030
201    44650
102    42624
       ...  
907        1
910        1
904        1
894        1
806        1
Name: catalog_nbr, Length: 1732, dtype: int64


In [ ]:
# Convert 'catalog_nbr' to numeric (ignore errors for non-numeric values)
df['catalog_nbr'] = pd.to_numeric(df['catalog_nbr'], errors='coerce')

# Filtering rows based on conditions
filtered_df = df[~((df['catalog_nbr'] >= 500) & (df['catalog_nbr'] % 10 == 5))]


len(filtered_df)

3547649

In [ ]:
len(df)

3574768